In [2]:
import os
import sys
import pandas as pd
import numpy as np
import PIL
import matplotlib.pyplot as plt

%matplotlib inline

seed = 16
np.random.seed(seed)

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


In [3]:
#check using system GPU for processing and declaring system/GPU parameters

from tensorflow.python.client import device_lib
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu
print(device_lib.list_local_devices())

# configure tensorflow before fitting model
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.Session(config=tf_config)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17086564416454884435
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1493781708
locality {
  bus_id: 1
}
incarnation: 8097066893979137004
physical_device_desc: "device: 0, name: GeForce GTX 860M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [4]:
# changing directory for flow_from_directory method
os.chdir('C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets')

In [5]:
num_classes = 120
batch_size = 12

train_datagen = ImageDataGenerator(rotation_range=15, shear_range=0.1, channel_shift_range=20,
                                    width_shift_range=0.1,  height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True,
                                    fill_mode='nearest', rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory('train', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)

validation_generator = validation_datagen.flow_from_directory('validation', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)


test_generator = test_datagen.flow_from_directory('test', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)


Found 9600 images belonging to 120 classes.
Found 2400 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [6]:
input_shape = (224, 224, 3)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import keras.utils
from keras.applications.imagenet_utils import decode_predictions
from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

In [7]:
from keras.applications import VGG16
# Load the VGG model
# choosing VG166 architecture initially as my hand-built models with 3x3 filters perform the best and generally is more popular

vgg16_base = Sequential()
vgg16_base.add(VGG16(weights='imagenet',
                  include_top=False,
                pooling='avg'))
vgg16_base.add(Dense(num_classes, activation='softmax'))
vgg16_base.layers[0].trainable = False

# compile
adam_op = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
vgg16_base.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(vgg16_base.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 120)               61560     
Total params: 14,776,248
Trainable params: 61,560
Non-trainable params: 14,714,688
_________________________________________________________________
None


In [10]:
vgg16_base.fit_generator(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=800, 
                    epochs=10, 
                    validation_steps=200,
                    callbacks=[early_stopping, tensorboard])

Epoch 1/10
800/800 [==============================] - 382s 478ms/step - loss: 5.0060 - acc: 0.0078 - val_loss: 4.9803 - val_acc: 0.0092
Epoch 2/10
800/800 [==============================] - 380s 475ms/step - loss: 4.9684 - acc: 0.0075 - val_loss: 4.9534 - val_acc: 0.0092
Epoch 3/10
800/800 [==============================] - 382s 477ms/step - loss: 4.9437 - acc: 0.0086 - val_loss: 4.9312 - val_acc: 0.0104
Epoch 4/10
800/800 [==============================] - 379s 474ms/step - loss: 4.9236 - acc: 0.0078 - val_loss: 4.9120 - val_acc: 0.0108
Epoch 5/10
800/800 [==============================] - 385s 481ms/step - loss: 4.9071 - acc: 0.0082 - val_loss: 4.8955 - val_acc: 0.0117
Epoch 6/10
800/800 [==============================] - 379s 474ms/step - loss: 4.8909 - acc: 0.0089 - val_loss: 4.8810 - val_acc: 0.0121
Epoch 7/10
800/800 [==============================] - 379s 474ms/step - loss: 4.8763 - acc: 0.0081 - val_loss: 4.8680 - val_acc: 0.0113
Epoch 8/10
800/800 [============================

In [11]:
vgg16_base.save('vgg16_base.h5')

In [16]:
base_scores = vgg16_base.evaluate_generator(test_generator, steps=25, max_queue_size=10)
print("Accuracy: %.2f%%" % (base_scores[1]*100))

Accuracy: 0.00%


In [8]:
from keras.models import Model
from keras.layers.normalization import BatchNormalization

In [9]:
base_model = VGG16(include_top=False, input_shape=input_shape)

# add a global spatial average pooling layer
x = base_model.output
x = Flatten()(x)
x = Dropout(0.4)(x)

# let's add a fully-connected layer

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)


predictions = Dense(num_classes, activation='softmax')(x)


model = Model(input=base_model.input, output=predictions)

# train only the top layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [10]:
model.fit_generator(train_generator, 
                    validation_data=validation_generator,
                    steps_per_epoch=800, 
                    epochs=10, 
                    validation_steps=200,
                    callbacks=[early_stopping])

Epoch 1/10
800/800 [==============================] - 394s 493ms/step - loss: 5.1161 - acc: 0.0083 - val_loss: 4.8716 - val_acc: 0.0158
Epoch 2/10
800/800 [==============================] - 394s 493ms/step - loss: 4.9540 - acc: 0.0099 - val_loss: 4.7744 - val_acc: 0.0183
Epoch 3/10
800/800 [==============================] - 394s 492ms/step - loss: 4.9082 - acc: 0.0103 - val_loss: 4.7551 - val_acc: 0.0200
Epoch 4/10
800/800 [==============================] - 394s 492ms/step - loss: 4.8774 - acc: 0.0143 - val_loss: 4.7454 - val_acc: 0.0225
Epoch 5/10
410/800 [==============>...............] - ETA: 2:34 - loss: 4.8587 - acc: 0.0104

KeyboardInterrupt: 